For the Questions 1-2, use the dataset MovieLens 100K.

Hints:

(1) The raw ids are ids that are defined in the rating file “u.data” in the MovieLens 100K
dataset.

(2) For movie names, refer to the file “u.item” in the MovieLens 100K dataset.

(3) Use the whole dataset for similarity calculation.

Dataset reference: https://grouplens.org/datasets/movielens/100k/

-----------------------------------------

### Question 1: about user-user similarity:

Task: Find out 5 users who are most similar to the user with the raw id of 243, and show
the raw ids of these 5 users. 

a. Compute user-user similarities by Pearson Correlation and obtain the result;

b. Compute user-user similarities by Cosine Similarity and obtain the result.

In [1]:
from surprise import Dataset
from surprise import Dataset
from surprise import Reader

import os
import pandas as pd
import numpy as np
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import accuracy
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
import io
from scipy import spatial
from scipy import stats
import math
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')
data.raw_ratings[:10] # first ten recoreds
data

In [3]:
# a) with pearson correlation
# Retrieve the trainset (whole dataset)
All_trainset = data.build_full_trainset()


In [4]:
# First, train the algortihm to compute the similarities between users
sim_options = {'name': 'pearson',
               'user_based': True,  # compute  similarities between users
               }
algo = KNNBasic(k=40,sim_options=sim_options) 

algo.fit(All_trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [5]:
def similar_users_with_pearson(top_k,u_id):
    
    # Retrieve inner id
    UserInnerId = algo.trainset.to_inner_uid(u_id)
    
    # Retrieve inner ids of the nearest neighbors usign the get_neighbors from surprise library
    UserNeighbors = algo.get_neighbors( UserInnerId, k=top_k)
    #Convert a user inner id to a raw id.
    UserNeighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in UserNeighbors)

    return UserNeighbors

list(similar_users_with_pearson(5,'243'))

['172', '191', '153', '103', '9']

In [6]:
#b) with cosine similarity
sim_options = {'name': 'cosine',
               'user_based': True,  # compute  similarities between users
               }
algo = KNNBasic(k=40,sim_options=sim_options) 

algo.fit(All_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [7]:
def similar_users_with_cosine(top_k,u_id):
    
    UserInnerId = algo.trainset.to_inner_uid(u_id)
    # Retrieve inner ids of the nearest neighbors usign the get_neighbors from surprise library
    UserNeighbors = algo.get_neighbors(UserInnerId, k=top_k)
    #Convert a user inner id to a raw id.
    UserNeighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in UserNeighbors)

    return UserNeighbors

list(similar_users_with_cosine(5,'243'))

['166', '127', '135', '229', '36']

### Question 2 about item-item similarity:

Task: Find out 5 movies that are most similar to the Movie named “Toy Story” (movie id: 1), and show the raw ids and the names of these 5 movies.

a. Compute item-item similarities by Pearson Correlation and obtain the result;

b. Compute item-item similarities by (basic) Cosine Similarity and obtain the result;

c. Compute item-item similarities by adjusted Cosine Similarity and obtain the result.

In [324]:
def read_item_names():

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

#### a) Pearson Correlaion

In [325]:
# First, train the algortihm to compute the similarities between items
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=5)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

List_neighbors = []
for movie in toy_story_neighbors:
    List_neighbors.append(movie)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [326]:
item_algo = KNNBasic(sim_options={'name': 'pearson','user_based': False}) 

item_algo.fit(trainset)

def similar_items_with_pearson(top_k,item_id):

    item_inner_id = item_algo.trainset.to_inner_iid(item_id)

    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)

    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)

    for inner_id in item_neighbors)

    return f_item_neighbors

List_id=list(similar_items_with_pearson(5,'1'))


Computing the pearson similarity matrix...
Done computing similarity matrix.


In [327]:
print('The 5 nearest neighbors of Toy Story with Pearson Correlation are:')
print()
for i in range(5):
    print(i+1,") ID:" ,List_id[i],"- Name of movie:",List_neighbors[i])

The 5 nearest neighbors of Toy Story with Pearson Correlation are:

1 ) ID: 1497 - Name of movie: Line King: Al Hirschfeld, The (1996)
2 ) ID: 1186 - Name of movie: Inkwell, The (1994)
3 ) ID: 1464 - Name of movie: Stars Fell on Henrietta, The (1995)
4 ) ID: 341 - Name of movie: Critical Care (1997)
5 ) ID: 1242 - Name of movie: Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)


#### b) Cosine sililarity

In [329]:
# First, train the algortihm to compute the similarities between items
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=5)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

List_neighbors = []
for movie in toy_story_neighbors:
    List_neighbors.append(movie)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [330]:
item_algo = KNNBasic(sim_options={'name': 'cosine','user_based': False}) 

item_algo.fit(trainset)

def similar_items_with_cosine(top_k,item_id):

    item_inner_id = item_algo.trainset.to_inner_iid(item_id)

    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)

    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)

    for inner_id in item_neighbors)

    return f_item_neighbors

List_iid=list(similar_items_with_cosine(5,'1'))

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [331]:
print('The 5 nearest neighbors of Toy Story with Cosine Similarity are:')
print()
for i in range(5):
    print(i+1,") ID:" ,List_iid[i],"- Name of movie:",List_neighbors[i])

The 5 nearest neighbors of Toy Story with Cosine Similarity are:

1 ) ID: 626 - Name of movie: So Dear to My Heart (1949)
2 ) ID: 1332 - Name of movie: My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993)
3 ) ID: 1334 - Name of movie: Somebody to Love (1994)
4 ) ID: 1350 - Name of movie: Crows and Sparrows (1949)
5 ) ID: 1260 - Name of movie: Total Eclipse (1995)


#### c) Adjusted Cosine

In [351]:
import numpy as np

In [352]:
# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')
train_set = data.build_full_trainset()
#print(trainset.to_inner_uid('186')) # here one will print 1
#print(trainset.to_raw_iid(1014))
#trainset.ur[0][1][1]  for user number zero the 1 row, 1 data (that is item id)
#trainset.ur[0] #it gives a list like this : uid: [(iid,rating)]

In [353]:
train_user_list = [u_id  for u_id in train_set.ur.keys()]
print(len(train_user_list))

train_item_list = [ i_id  for i_id in train_set.ir.keys()]
print(len(train_item_list))

943
1682


In [354]:
# calculate adjusted similarity between two items
import math
import numpy as np

# initiate the matrix for storing the similarity values, (1682*1682)
sim_item = np.empty((len(train_item_list),len(train_item_list)))

# calculateing the mean rating of each user
u_rating_mean= {}
for u_id in range(len(train_user_list)):
    sum_r = 0
    count = 0
    for (i,r) in train_set.ur[u_id]:
        sum_r += r
        count += 1
    u_rating_mean[u_id] = sum_r / count # for each user,this dict retuens its mean rating
    
# loop through all items to calculate the similaity
for i1_id in range(len(train_item_list)):
    for i2_id in range(i1_id+1,len(train_item_list)):
        
        # A: find common users that rated to both i1 and i2
        i1_user_rating_dict = {}     
        for (i, r) in train_set.ir[i1_id]:
            i1_user_rating_dict[i] = r
        i2_user_rating_dict = {}
        for (i, r) in train_set.ir[i2_id]:
            i2_user_rating_dict[i] = r
        common_users = list(set.intersection(set(i1_user_rating_dict.keys()), \
                                             set(i2_user_rating_dict.keys())))
        
        
        # B: construct rating vectors of common users for two items
        i1_rating_list = []
        i2_rating_list = []
        commonuser_rating_mean = []   # I have created an empty list to access the average ranking of common users
        for i in common_users:
            i1_rating_list.append(i1_user_rating_dict[i]) 
            i2_rating_list.append(i2_user_rating_dict[i]) 
            commonuser_rating_mean.append(u_rating_mean[i])
            
        dif_i1_and_mean = []
        for i in range(len(i1_rating_list)):
            d = i1_rating_list[i] - commonuser_rating_mean[i]
            dif_i1_and_mean.append(d)
  
        dif_i2_and_mean = []
        for i in range(len(i2_rating_list)):
            d2 = i2_rating_list[i] - commonuser_rating_mean[i]
            dif_i2_and_mean.append(d2)
    
        # C: calculate the similarities between two items 
        # - we used adjusted cosine similairty here
        
        if len(common_users) > 0:
            adjcos_sim = 1 - spatial.distance.cosine( dif_i1_and_mean , dif_i2_and_mean)
        else: 
            adjcos_sim = 0
            
    
        # D: store measured similarity into matrix
        sim_item[i1_id][i2_id] = adjcos_sim
        sim_item[i2_id][i1_id] = adjcos_sim

        
#After completing the calculation, store the similarity matrix into a table
sim_matrix_table = pd.DataFrame(sim_item)
print(sim_matrix_table.head(5))
sim_matrix_table.to_csv('item_similarty_table.csv') 
 

       0         1         2         3         4         5         6     \
0  0.000000  0.459114  1.000000 -0.384484  0.616092  0.162003  0.120053   
1  0.459114  0.000000 -0.715449 -0.102312  0.405081  0.622343  0.288585   
2  1.000000 -0.715449  0.000000  0.174535 -1.000000 -0.909653 -0.321567   
3 -0.384484 -0.102312  0.174535  0.000000 -0.240694 -0.148113  0.035582   
4  0.616092  0.405081 -1.000000 -0.240694  0.000000  0.344501  0.118103   

       7         8         9     ...      1672  1673  1674  1675  1676  1677  \
0 -0.416933  0.246056  0.447064  ...  0.847661   0.0   1.0   0.0   0.0   0.0   
1 -0.109022  0.020597  0.171639  ...  0.728160   0.0   1.0  -1.0   0.0   0.0   
2  0.549063  0.139658  0.401550  ...  0.000000   1.0   0.0   0.0   0.0   0.0   
3 -0.251833 -0.245013 -0.239582  ...  0.728160   0.0  -1.0   0.0   1.0   0.0   
4  0.434372 -0.341546  0.099995  ...  0.069474   0.0   1.0   0.0   0.0   0.0   

   1678  1679  1680  1681  
0   1.0   1.0   1.0   1.0  
1   1.0   1.

In [355]:
raw_item_id = str(1)
k = 5

item_id = train_set.to_inner_iid(raw_item_id)
print(item_id)

neighbors = []
neighbors_sim_dict = {}
for u in train_item_list:
    if item_id != u:
        neighbors.append(u)
        neighbors_sim_dict[u] = sim_item[item_id, u]
    

# get K nearest neighbors
k_neighbors = sorted(neighbors_sim_dict.items(), reverse=True, key=lambda item: item[1])[:k]
print(k_neighbors)

List_row= [ ]
for (i,r) in k_neighbors:
    List_row.append(train_set.to_raw_iid(i))

print("With row item id we have:",List_row)


24
[(756, 1.0), (941, 1.0), (962, 1.0), (1033, 1.0), (1038, 1.0)]
With row item id we have: ['1436', '1493', '1464', '830', '1189']


In [356]:
List_row

['1436', '1493', '1464', '830', '1189']

In [357]:
toy_story_neighbors = (rid_to_name[rid]
                       for rid in List_row)

for movie in toy_story_neighbors:
    print(movie)

Mr. Jones (1993)
Modern Affair, A (1995)
Stars Fell on Henrietta, The (1995)
Power 98 (1995)
Prefontaine (1997)


### Question 3 :
Task: Conduct a comparison between centered user-based k-NN (with means) and
centered item-based k-NN (with means) on the Jester dataset (see the attached dataset
Jester_200.zip). Note that you need to load the data file jester_ratings_200.dat first by
using ‘Dataset.load_from_file()’ in Surprise.

a. Use Grid Search to search for the best combination of similarity measure and k value
for each method (metric is RMSE);

b. Use the 10-fold cross-validation to test the best combination for each method;

c. Compare the two methods’ results through Two-Sample t-Test;

d. Briefly describe the results and discuss the two methods’ performance on this dataset.m

In [136]:
from surprise import Dataset
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
import os
from surprise import Dataset
from surprise import Reader

# path to dataset file
file_path = os.path.expanduser('./jester_ratings_200.dat')
reader = Reader(line_format='user item rating', sep='\t\t')
data = Dataset.load_from_file(file_path, reader=reader)
data.raw_ratings[:10] # first ten recoreds

[('162', '5', 3.719, None),
 ('162', '7', 9.75, None),
 ('162', '8', 7.469, None),
 ('162', '13', -9.812, None),
 ('162', '15', 2.312, None),
 ('162', '16', 5.031, None),
 ('162', '17', 0.469, None),
 ('162', '18', -2.594, None),
 ('162', '19', 5.531, None),
 ('162', '20', -1.688, None)]

In [129]:
# a,b) 
gs_cv = 3
eval_cv = 10

# Use Centered KNN (user-based) algorithms
user_based_param_grid = {'k': [10, 20, 60, 90, 125, 200],
              'sim_options':{
                  'name': ['cosine','pearson'],
                  'user_based': [True]
                  }
             }

# Find out the best combination of similarity measurement and k value 
user_based_gs = GridSearchCV(KNNWithMeans, user_based_param_grid, measures=['rmse'], cv=gs_cv)
user_based_gs.fit(data)
user_based_algo = user_based_gs.best_estimator['rmse']

# Use cross validation to evaluate the best model
user_best_result = cross_validate(user_based_algo, data, measures=['rmse'], cv=eval_cv, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [130]:
# Use Centered KNN (item-based) algorithms

# Find out the best combination of similarity measurement and k value 
item_based_param_grid = {'k': [10, 20, 60, 90, 125, 200],
              'sim_options':{
                  'name': ['cosine','pearson'],
                  'user_based': [False]
                  }
             }
item_based_gs = GridSearchCV(KNNWithMeans, item_based_param_grid, measures=['rmse'], cv=gs_cv)
item_based_gs.fit(data)

item_based_algo = item_based_gs.best_estimator['rmse']

# Use cross validation to evaluate the best model
item_best_result = cross_validate(item_based_algo, data, measures=['rmse'], cv=eval_cv,verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [131]:
user_based_gs.best_params['rmse']

{'k': 60, 'sim_options': {'name': 'pearson', 'user_based': True}}

In [132]:
item_based_gs.best_params['rmse']

{'k': 20, 'sim_options': {'name': 'cosine', 'user_based': False}}

In [133]:
# c) T-Test
from scipy.stats import ttest_ind

user_based_RMSE = user_best_result['test_rmse']
item_based_RMSE = item_best_result['test_rmse']

print(user_based_RMSE)
print(item_based_RMSE)

ttest_ind(user_based_RMSE,item_based_RMSE)

[4.33534511 4.54114223 4.37929032 4.49299361 4.3335028  4.56022417
 4.39520036 4.47050553 4.40618164 4.45927413]
[4.47070944 4.33863671 4.51196758 4.3952563  4.40670497 4.48424168
 4.42120875 4.32521619 4.36903056 4.39429541]


Ttest_indResult(statistic=0.8020081775944554, pvalue=0.43300956897443577)

d)

If  𝑝  <0.05, it means significant difference between two methods.

Conclusion:

For Jester_200, the result showed that there is no significant difference between item-based centered k-NN and user_based centered k-NN.

The mean of user_based centered k-NN is 4.4374 and the mean of item-based centered k-NN is 4.4117 which are close to each other and it was confirmed by T-test that these differences are not significant. 

We also showed that the best parameter combination for the user_based centered k-NN is to use k =60 and pearson correlation as measurment similarity, and the best parameter combination for the item_based centered k-NN is to use k =20 and cosine similarity as measurment similarity.